In [23]:
import pandas as pd
import pickle
import numpy as np

In [24]:
df = pickle.load(open('innings1.pkl', 'rb'))
df

,batting_team,bowling_team,city,cumulative_runs,ballsleft,wicketsleft,rr,last5,runs_x,cumulative_wickets,last5w
0,Australia,Sri Lanka,Melbourne,0,119,10,0.000000,NaN,168,0,NaN
1,Australia,Sri Lanka,Melbourne,0,118,10,0.000000,NaN,168,0,NaN
2,Australia,Sri Lanka,Melbourne,1,117,10,2.000000,NaN,168,0,NaN
3,Australia,Sri Lanka,Melbourne,3,116,10,4.500000,NaN,168,0,NaN
4,Australia,Sri Lanka,Melbourne,3,115,10,3.600000,NaN,168,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
75006,Sri Lanka,Australia,Colombo,125,3,10,6.410256,32.0,128,8,1.0
75007,Sri Lanka,Australia,Colombo,125,2,10,6.355932,32.0,128,8,1.0
75008,Sri Lanka,Australia,Colombo,125,1,9,6.302521,32.0,128,9,2.0
75009,Sri Lanka,Australia,Colombo,127,0,10,6.350000,33.0,128,9,2.0


In [25]:
df['wicketsleft'] = df['cumulative_wickets'].apply(lambda x: 10-x)

In [26]:
df.dropna(inplace = True)

In [27]:
df


,batting_team,bowling_team,city,cumulative_runs,ballsleft,wicketsleft,rr,last5,runs_x,cumulative_wickets,last5w
29,Australia,Sri Lanka,Melbourne,43,90,10,8.600000,43.0,168,0,0.0
30,Australia,Sri Lanka,Melbourne,44,89,10,8.516129,44.0,168,0,0.0
31,Australia,Sri Lanka,Melbourne,45,88,10,8.437500,45.0,168,0,0.0
32,Australia,Sri Lanka,Melbourne,45,87,10,8.181818,44.0,168,0,0.0
33,Australia,Sri Lanka,Melbourne,45,86,10,7.941176,42.0,168,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
75006,Sri Lanka,Australia,Colombo,125,3,2,6.410256,32.0,128,8,1.0
75007,Sri Lanka,Australia,Colombo,125,2,2,6.355932,32.0,128,8,1.0
75008,Sri Lanka,Australia,Colombo,125,1,1,6.302521,32.0,128,9,2.0
75009,Sri Lanka,Australia,Colombo,127,0,1,6.350000,33.0,128,9,2.0


In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf = ColumnTransformer([
    ('trf', OneHotEncoder(sparse_output=False, drop='first'), ['batting_team', 'bowling_team', 'city'])
], remainder='passthrough')

In [29]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [30]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=300,learning_rate=0.1,max_depth=12,random_state=1))
])

In [31]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['runs_x'])
y = df['runs_x']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [32]:
from sklearn.metrics import r2_score,mean_absolute_error

In [33]:
X

,batting_team,bowling_team,city,cumulative_runs,ballsleft,wicketsleft,rr,last5,cumulative_wickets,last5w
29,Australia,Sri Lanka,Melbourne,43,90,10,8.600000,43.0,0,0.0
30,Australia,Sri Lanka,Melbourne,44,89,10,8.516129,44.0,0,0.0
31,Australia,Sri Lanka,Melbourne,45,88,10,8.437500,45.0,0,0.0
32,Australia,Sri Lanka,Melbourne,45,87,10,8.181818,44.0,0,0.0
33,Australia,Sri Lanka,Melbourne,45,86,10,7.941176,42.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
75006,Sri Lanka,Australia,Colombo,125,3,2,6.410256,32.0,8,1.0
75007,Sri Lanka,Australia,Colombo,125,2,2,6.355932,32.0,8,1.0
75008,Sri Lanka,Australia,Colombo,125,1,1,6.302521,32.0,9,2.0
75009,Sri Lanka,Australia,Colombo,127,0,1,6.350000,33.0,9,2.0


In [34]:
y

29       168
30       168
31       168
32       168
33       168
        ... 
75006    128
75007    128
75008    128
75009    128
75010    128
Name: runs_x, Length: 57129, dtype: int64

In [35]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.9874318838119507
1.913278819424205


In [36]:
pickle.dump(pipe, open('final_model1.pkl', 'wb'))